In [1]:
""" neural network """
# importing packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from pickle import dump, load
from scipy.stats import pearsonr

In [2]:
# helper functions
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
# visualize data
df = pd.read_csv('../../data/TRAIN.csv')
df2 = pd.read_csv('/Users/SnehPandya/Desktop/nn/train_drw_full.csv')
df.iloc[1,14:23]
df2.iloc[1,14:23]

z         0.479000
M_i     -24.046000
ug        0.238531
gr       -0.050367
ri        0.082918
iz        0.077531
zu       -0.348612
tau       2.227110
sigma    -0.660666
Name: 1, dtype: float64

In [4]:
# Black Hole dataset--default parameters are for training and predicting AGN mass.  Pass 'train=False' 
# for test-set and 'mass=False' for AGN redshift prediction.
class BHDataset(Dataset):
    def __init__(self, path, train=True, mass=True):
        self.path = path
        self.train = train
        self.mass = mass
        self.sc = StandardScaler()
        
        if self.mass:
            
            if self.train:
                self.data = pd.read_csv(self.path + 'TRAIN.csv')
                self.features = self.sc.fit_transform(np.asarray(self.data.iloc[:,14:23]))
                dump(self.sc, open('train_scaler_drw_full.pkl','wb'))
        
            else:
                self.data = pd.read_csv(self.path + 'TEST.csv')
                self.sc = load(open('train_scaler_drw_full.pkl','rb'))
                self.features = self.sc.transform(np.asarray(self.data.iloc[:,14:23]))
                
        else:
            
            if self.train:
                self.data = pd.read_csv(self.path + 'TRAIN.csv')
                self.features = self.sc.fit_transform(np.asarray(self.data.iloc[:,[9,10,11,12,13,16,17,18,19,20]]))
                dump(self.sc, open('train_scaler_drw_full.pkl','wb'))
        
            else:
                self.data = pd.read_csv(self.path + 'TEST.csv')
                self.sc = load(open('train_scaler_drw_full.pkl','rb'))
                self.features = self.sc.transform(np.asarray(self.data.iloc[:,[9,10,11,12,13,16,17,18,19,20]]))
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        if self.mass:
            
            ID = torch.from_numpy(np.asarray(self.data.iloc[index,0]))
            target = torch.from_numpy(np.asarray(self.data.iloc[index,1]))
            features = torch.from_numpy(self.features[index].reshape(1,-1).squeeze())
            return (ID, features, target)
        
        else:
            ID = torch.from_numpy(np.asarray(self.data.iloc[index,0]))
            target = torch.from_numpy(np.asarray(self.data.iloc[index,14]))
            features = torch.from_numpy(self.features[index].reshape(1,-1).squeeze())
            return (ID, features, target)

        
# define train and test datasets.  Train test split was done previously using sklearn.
train_mass = BHDataset('../../data/')
test_mass = BHDataset('../../data/', train=False)

train_z = BHDataset('../../data/', mass=False)
test_z = BHDataset('../../data/', mass=False, train=False)

In [5]:
# Define dataloaders with the datasets.  Only shuffle training sets.
train_dl_mass = DataLoader(train_mass, batch_size=100, shuffle=True)
test_dl_mass = DataLoader(test_mass, batch_size=100, shuffle=False)

train_dl_z = DataLoader(train_z, batch_size=100, shuffle=True)
test_dl_z = DataLoader(test_z, batch_size=100, shuffle=False)

In [11]:
# default architecture is to predict AGN mass.  Pass 'mass=False' to predict redshift.
class AGNet(nn.Module):
    def __init__(self, mass=True):
        super().__init__()
        self.mass = mass
        
        if self.mass:

            self.fc1 = nn.Linear(9, 32)
            self.fc2 = nn.Linear(32, 64)
            self.fc3 = nn.Linear(64, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            
        else:
            
            self.fc1 = nn.Linear(10, 32)
            self.fc2 = nn.Linear(32, 64)
            self.fc3 = nn.Linear(64, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

# defining neural networks
net_mass = AGNet()
net_z = AGNet(mass=False)

In [12]:
# count parameters for reference.  For both redshift and mass, (training size/model parameters) < 1.
(count_parameters(net_mass), count_parameters(net_z))

(8705, 8737)

In [13]:
# loss function and optimizer -- change argument of optimizer to correct network depending on prediction
lr = .01
optimizer = optim.AdamW(net_mass.parameters(), lr=lr)
loss_function = F.smooth_l1_loss

In [14]:
# training loop takes in epoch #, network, train loader, loss function, and optimizer.  
# Plots RMSE pr epoch and returns RMSE/loss vs. epoch plots at end
def train(num_epochs, trainloader, testloader, mdl):
    
    epoch_list = np.linspace(1, num_epochs , num = num_epochs)
    loss_list, rmse_list = list(), list()

    for epoch in range(num_epochs):

            train_pred, train_gt, test_pred, test_gt = list(), list(), list(), list()

            for data in trainloader:

                ID, features, ground_truth = data
                train_gt.append(ground_truth.float())
                output_train = mdl(features.float())
                train_pred.append(output_train.float())
                train_loss = loss_function(output_train.squeeze(), ground_truth.float().squeeze())
                mdl.zero_grad()
                train_loss.backward()
                optimizer.step()

            train_ground_truth = torch.cat(train_gt).data
            train_predictions = torch.cat(train_pred).data.flatten()
            train_rmse = np.sqrt(metrics.mean_squared_error(train_ground_truth, train_predictions))

            with torch.no_grad():
                for data_ in testloader:  

                    ID_, features_, ground_truth_ = data_  
                    test_gt.append(ground_truth_.float())
                    output_test = mdl(features_.float()) 
                    test_pred.append(output_test.float())
                    test_loss = loss_function(output_test.squeeze(), ground_truth_.float())
                    test_rmse = np.sqrt(metrics.mean_squared_error(output_test.squeeze().detach().numpy(), ground_truth_.float().squeeze().detach().numpy()))

                test_ground_truth = torch.cat(test_gt).data
                test_predictions = torch.cat(test_pred).data.flatten()
                test_rmse = np.sqrt(metrics.mean_squared_error(test_ground_truth, test_predictions))

                print(f'EPOCH: {epoch+1}\nTrain Loss: {train_loss.item():.5f} | Train RMSE: {train_rmse:.5f}\nTest Loss: {test_loss.item():.5f} | Test RMSE: {test_rmse:.5f}')

In [15]:
train(50, train_dl_mass, test_dl_mass, net_mass)

EPOCH: 1
Train Loss: 0.11561 | Train RMSE: 2.71970
Test Loss: 0.08652 | Test RMSE: 0.47074
EPOCH: 2
Train Loss: 0.07911 | Train RMSE: 0.45523
Test Loss: 0.06912 | Test RMSE: 0.43378
EPOCH: 3
Train Loss: 0.08644 | Train RMSE: 0.42195
Test Loss: 0.06493 | Test RMSE: 0.39172
EPOCH: 4
Train Loss: 0.09880 | Train RMSE: 0.41829
Test Loss: 0.07936 | Test RMSE: 0.41992
EPOCH: 5
Train Loss: 0.11981 | Train RMSE: 0.43280
Test Loss: 0.07290 | Test RMSE: 0.41174
EPOCH: 6
Train Loss: 0.07620 | Train RMSE: 0.40724
Test Loss: 0.07111 | Test RMSE: 0.40638
EPOCH: 7
Train Loss: 0.07872 | Train RMSE: 0.41075
Test Loss: 0.05296 | Test RMSE: 0.37471
EPOCH: 8
Train Loss: 0.06451 | Train RMSE: 0.40895
Test Loss: 0.08424 | Test RMSE: 0.46292
EPOCH: 9
Train Loss: 0.12743 | Train RMSE: 0.40797
Test Loss: 0.09759 | Test RMSE: 0.48624
EPOCH: 10
Train Loss: 0.15242 | Train RMSE: 0.43338
Test Loss: 0.09600 | Test RMSE: 0.45413
EPOCH: 11
Train Loss: 0.10797 | Train RMSE: 0.41379
Test Loss: 0.06112 | Test RMSE: 0.381

The current combination of number of epochs, loss function, and optimizer are what we found to work best.  We encourage users to explore other combinations!

In [ ]:
# save model
# torch.save(net_mass.state_dict(), '../../data/AGNet.mdl')

In [ ]:
# test loop outputs plot of results + dataframe of object ID, ground truth values, and network predictions
def test(testloader, mdl):

    with torch.no_grad():
        
        test_ID, test_pred, test_gt = list(), list(), list()

        for data in testloader:
            ID, features, ground_truth = data  
            test_ID.append(ID.float())
            test_gt.append(ground_truth.float())
            output_test = mdl(features.float()) 
            test_pred.append(output_test.float())
            test_loss = loss_function(output_test.squeeze(), ground_truth.float())

        test_ground_truth = torch.cat(test_gt).data
        test_predictions = torch.cat(test_pred).data.flatten()
        test_rmse = np.sqrt(metrics.mean_squared_error(test_ground_truth, test_predictions))
        ID = torch.cat(test_ID).data

        plt.plot(test_ground_truth, test_ground_truth,color='black', label = 'Mass Ground Truth')
        plt.scatter(test_ground_truth, test_predictions,s=2, color='blue', label = 'NN prediction')
        plt.title('RMSE:' + str(test_rmse) + ' | LOSS: ' + str(test_loss.data.numpy()))
        plt.xlabel('AGN Mass')
        plt.ylabel('AGN Mass')
        plt.legend()
        plt.show()
        
        df = pd.DataFrame({'ID':ID.numpy(), 'ground truth':test_ground_truth.numpy(), 'network predictions':test_predictions.numpy() })
        return df

In [ ]:
df = test(test_dl_mass, net_mass)

In [ ]:
# output test-set results to csv
df.to_csv('../../data/AGNet_results.csv')